<a href="https://colab.research.google.com/github/Teuteu666/ANN_Optimized_CS/blob/main/mini_projetRO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
################################################################################
#                                                                              #
#	BELGHADI MOHAMMED                                                            #
#	CSO (CUCKOO SEARCH OPTIMIZATION)                                             #
#	MNIST(ANN)                                                                   #
#                                                                              #
################################################################################

import numpy as np
import time
import matplotlib.pyplot as plt
import random

train_file = open("/content/drive/MyDrive/CS_DataSet/mnist_train.csv", 'r')
test_file = open("/content/drive/MyDrive/CS_DataSet/mnist_test.csv", 'r')
train_list = train_file.readlines()
test_list = test_file.readlines()

train_file.close()
test_file.close()

def saveWeigths(dic:dict,filename):
    for k,v in dic.items():
        if k=="W1" or k=="W2" or k=="W3" :
            np.save(filename+"/"+k+".npy",v)

class DNN:
    def __init__(self, sizes=[128,64],epochs=2, lr=0.01):
        self.sizes = sizes
        self.epochs = epochs
        self.lr = lr

        input_layer = 784
        self.hidden_1 = sizes[0]
        self.hidden_2 = sizes[1]
        output_layer = 10

        self.params = {
            "W1": np.random.randn(self.hidden_1, input_layer) * np.sqrt(1 / self.hidden_1),
            "W2": np.random.randn(self.hidden_2, self.hidden_1) * np.sqrt(1 / self.hidden_2),
            "W3": np.random.randn(output_layer, self.hidden_2) * np.sqrt(1 / output_layer)
        }

    def loadWeight(self,filename):
    # the filename should mention the extension 'npy'
        tempNumpyArray=np.load(filename)
        return tempNumpyArray
    def sigmoid(self, x, derivation=False):
        # the flag is used in backwards
        if derivation:
            return (np.exp(-x))/((np.exp(-x)+1)**2)
        else:
            return 1/(np.exp(-x)+1)

    def softmax(self, x, derivation=False):
        exps = np.exp(x-x.max())
        if derivation:
            return exps/np.sum(exps, axis=0) * (1-exps/np.sum(exps, axis=0))
        return exps/np.sum(exps, axis=0)

    def forward_pass(self, x_train):
        # using a copy is to keep list of weights for backpropagation
        params = self.params
        params["A0"] = x_train
        # from input_layer to hidden_1
        params["Z1"] = np.dot(params["W1"], params["A0"])
        params["A1"] = self.sigmoid(params["Z1"])
        # from hidden_1 to hidden_2
        params["Z2"] = np.dot(params["W2"], params["A1"])
        params["A2"] = self.sigmoid(params["Z2"])
        # from hidden_2 to output_layer
        params["Z3"] = np.dot(params["W3"], params["A2"])
        params["A3"] = self.softmax(params["Z3"])
        return params['A3']

    def backward_pass(self, y_train, output,v):
        params = self.params
        change_w = {}
        # calculate w3 update
        error = 2 * (output - y_train) / output.shape[0] * self.softmax(params["Z3"])
        change_w['W3'] = np.outer(error, params["A2"])
        # calculate w2 update
        error = np.dot(params['W3'].T, error) * self.sigmoid(params["Z2"], derivation=True)
        change_w['W2'] = np.outer(error, params["A1"])
        # calculate w1 update
        error = np.dot(params['W2'].T, error) * self.sigmoid(params["Z1"], derivation=True)
        change_w['W1'] = np.outer(error, params["A0"])

        v["W3"] = 0.9*v["W3"] + change_w["W3"]
        v["W2"] = 0.9*v["W2"] + change_w["W2"]
        v["W1"] = 0.9*v["W1"] + change_w["W1"]
        return v

    def update_weights(self, change_w: dict):
        for key, val in change_w.items():
            self.params[key] -= self.lr*val

    def compute_accuracy(self, test_data):
        predictions = []
        for x in test_data:
            values = x.split(',')
            inputs = (np.asfarray(values[1:])/255.0*0.99)+0.01
            targets = np.zeros(10)+0.01
            targets[int(values[0])] = 0.99
            output = self.forward_pass(inputs)
            pred = np.argmax(output)
            predictions.append(pred == np.argmax(targets))
        return np.mean(predictions)

    def train(self, train_list, test_list):
        v={
            "W1": np.zeros((self.hidden_1, 784)),
            "W2": np.zeros((self.hidden_2, self.hidden_1)),
            "W3": np.zeros((10, self.hidden_2))
        }
        for i in range(self.epochs):
            start_time = time.time()
            for x in train_list:
                # plsit
                values = x.split(',')

                inputs = (np.asfarray(values[1:]) / 255.0 * 0.99) + 0.01

                targets = np.zeros(10)+0.01
                targets[int(values[0])] = 0.99
                output = self.forward_pass(inputs)
                v = self.backward_pass(targets, output, v)
                self.update_weights(v)
            accuracy = self.compute_accuracy(test_list)
            print("epoch:",i+1,"accuracy",accuracy*100,"%")
        return accuracy





In [ ]:

import numpy  as np
import matplotlib.pyplot as plt
from math import gamma
import glob
from PIL import Image

class CSO:

    def __init__(self,x_train,y_train,sizes=[128, 64], epochs=3,population=3,beta=1.5,pa=0.25,plot=False):

        # ANN parameters
        self.x_train = x_train
        self.y_train = y_train
        self.sizes = sizes
        self.epochs = epochs

        #cs parameter Initialization
        self.population = population
        self.beta = beta
        self.pa=pa
        self.plot=plot
        self.nestLength=784*self.sizes[0] + self.sizes[0]*self.sizes[1]+ self.sizes[1]*10+3
        #nest Initialization
        self.nests=self.initNests()
        #init fitnesses
        self.fitnesses=self.initFitnesses()
        #init la meileur solution
        self.bestIndice=np.argmax(self.fitnesses)
        self.best=self.nests[self.bestIndice]

    def sigmoid(self, x):
            return 1/(np.exp(-x)+1)

    def softmax(self, x, derivation=False):
        exps = np.exp(x-x.max())
        return exps/np.sum(exps, axis=0)
    def relu(x):
      return np.max(0, x,axis=0)

    def loadNest(self,filename):
        tempNumpyArray=np.load(filename)
        return tempNumpyArray

    def saveNests(self):
      for i,n in enumerate(self.nests):
          np.save("/content/drive/MyDrive/CS_DataSet/N"+str(i),n)
      print("nests are saved")

    #initializing weights and biases d wahd nest
    def initWeights(self):
        input_layer = 784
        hidden_1 = self.sizes[0]
        hidden_2 = self.sizes[1]
        output_layer = 10
        params = {
            "W1": np.random.randn(hidden_1, input_layer)*np.sqrt(1./hidden_1),
            "W2": np.random.randn(hidden_2, hidden_1)*np.sqrt(1./hidden_2),
            "W3": np.random.randn(output_layer, hidden_2)*np.sqrt(1./output_layer),
            "B": np.random.randn(3)
        }
        tmp=np.array([])

        layer1_weights= params["W1"].flatten()
        tmp_a=np.append(tmp,layer1_weights)

        layer2_weights= params["W2"].flatten()
        tmp_b=np.append(tmp_a,layer2_weights)

        layer3_weights= params["W3"].flatten()
        tmp_c=np.append(tmp_b,layer3_weights)

        biasies= params["B"].flatten()

        nest=np.append(tmp_c,biasies)

        return nest

    #initializing nests
    def initNests(self):
        nests=np.zeros((self.population,self.nestLength))
        """n1=self.loadNest("/content/drive/MyDrive/CS_DataSet/N0.npy")
        n2=self.loadNest("/content/drive/MyDrive/CS_DataSet/N1.npy")
        n3=self.loadNest("/content/drive/MyDrive/CS_DataSet/N4.npy")"""
        for i in range(self.population):
            #initializing kula nest b weights dialu
            nests[i]=self.initWeights()
            """if i % 3 == 0:
              nests[i] = n1
              print("nest"+str(i)+" is initialized")
            elif i % 3 == 1:
              print("nest"+str(i)+" is initialized")
              nests[i] = n2
            else:
              print("nest"+str(i)+" is initialized")
              nests[i]=n3"""
        return nests
    def initFitnesses(self):
        fitnesses=[]
        for p in range(self.population):
            fitnesses.append(self.getAccuracyPerNest(self.nests[p]))
        return np.array(fitnesses)

    def update_position_1(self):
        #POUR CALCULER LE CHANGEMENT DE POSITION 'XNew = X + rand*C' EN UTILISANT LA MÉTHODE LEVY FLIGHT
        # C = 0.01*S*(X-best) WHERE S IS THE RANDOM STEP, and β = beta
        num = gamma(1+self.beta)*np.sin(np.pi*self.beta/2)
        den = gamma((1+self.beta)/2)*self.beta*(2**((self.beta-1)/2))

        #                     Γ(1+β)*sin(πβ/2)
        #       σu = --------------------------
        #               Γ((1+β)/2)*β*(2^((β-1)/2))

        sigma_u = (num/den)**(1/self.beta)
        #σv = 1
        sigma_v = 1
        #u ~ N(0,σu)  DISTRIBUTION NORMALE AVEC DÉVIATION STANDARD 'σu'.
        u = np.random.normal(0, sigma_u, self.nestLength)
        #v ~ N(0,σv)  DISTRIBUTION NORMALE AVEC DÉVIATION STANDARD 'σv'.
        v = np.random.normal(0, sigma_v, self.nestLength)
        #       u
        # S = -----
        #        1/β
        #      |v|
        S = u/(np.abs(v)**(1/self.beta))


        self.bestIndice=np.argmax(self.fitnesses)
        self.best=self.nests[self.bestIndice]

        #GENERATION DE NOUVEAUX SOLUTION ET GARDER LA MEILLEUR FITNESS
        newNest = self.nests.copy()
        for i in range(self.population):
          #GENERATION DE NEVAUX SOLUTION
            newNest[i,:] += np.random.randn(self.nestLength)*0.01*S*(newNest[i,:]-self.best)
            #CHOX DU MEILLEURS
            self.nests[i,:] = self.optimum(newNest[i,:],i)

    def update_position_2(self):
        #HÔTE PEUT JETER L'ŒUF (ABANDONNER LE NID) AVEC UNE FRACTION
        #pa ∈ [0,1] ET CONSTRUIRE UN NOUVEAU NID COMPLÈTEMENT
            Xnew = self.nests.copy()
            Xold = self.nests.copy()
            for i in range(self.population):
                #generate juj random indices within the population
                d1,d2 = np.random.randint(0,self.population,2)

                for j in range(self.nestLength):
                  #GENEATION D4UN NB ALEATOIRE
                    r = np.random.rand()
                    if r < self.pa:
                      #GENERATION DE NVX SOL
                        Xnew[i,j] += np.random.rand()*(Xold[d1,j]-Xold[d2,j])
                self.nests[i,:] = self.optimum(Xnew[i,:],i)
    #pour determiner le nest ayant
    def optimum(self,newNest,indiceOldNest):
            newFitness=self.getAccuracyPerNest(newNest)
            oldFitness=self.getFitness(indiceOldNest)

            #INIT OPTIMUM
            best=self.nests[indiceOldNest]

            if oldFitness < newFitness:
                best = newNest.copy()
                #MISE A JOUR DE FITNESS
                self.fitnesses[indiceOldNest]=newFitness
            return best
    def getFitness(self,indice):
            return self.fitnesses[indice]
    def getAccuracyPerNest(self,nest):
        params ={
                #sizes ..
                "W1": nest[:self.sizes[0] * 784].reshape(( self.sizes[0] , 784) ),
                "W2": nest[self.sizes[0] * 784 : self.sizes[0] * 784 + self.sizes[1] * self.sizes[0]].reshape((self.sizes[1],self.sizes[0])),
                "W3": nest[self.sizes[0]*784+self.sizes[1] *self.sizes[0]:self.sizes[0]*784+self.sizes[1] *self.sizes[0]+self.sizes[1]*10].reshape((10,self.sizes[1])),
                "B" : nest[-3:].reshape(3,)
            }
        ls_predictions=[]
        #forward pass
        for i,x in enumerate(self.x_train):
            params["A0"] = x
            # from input_layer to hidden_1
            params["Z1"] = np.dot(params["W1"], params["A0"]) + params["B"][0]
            params["A1"] = self.sigmoid(params["Z1"])
            # from hidden_1 to hidden_2
            params["Z2"] = np.dot(params["W2"], params["A1"])+ params["B"][1]
            params["A2"] = self.sigmoid(params["Z2"])
            # from hidden_2 to output_layer
            params["Z3"] = np.dot(params["W3"], params["A2"])+ params["B"][2]
            params["A3"] = self.softmax(params["Z3"])
            # Get the indices of the highest probability predictions
            prediction_indice = np.argmax(params["A3"])
            label = self.y_train[i]
            # Get the indices of the true labels
            label_indice = np.argmax(label)
            # Compare predictions and labels
            correct_prediction = np.equal(prediction_indice, label_indice)
            ls_predictions.append(correct_prediction)
        accuracy = np.mean(ls_predictions)
        return accuracy

    def make_prediction(self,images):
        #recuperation des weights ayant la meilleure fitness
        params ={

                "W1": self.best[:self.sizes[0] * 784].reshape(( self.sizes[0] , 784) ),
                "W2": self.best[self.sizes[0] * 784 : self.sizes[0] * 784 + self.sizes[1] * self.sizes[0]].reshape((self.sizes[1],self.sizes[0])),
                "W3": self.best[self.sizes[0]*784+self.sizes[1] *self.sizes[0]:self.sizes[0]*784+self.sizes[1] *self.sizes[0]+self.sizes[1]*10].reshape((10,self.sizes[1])),
                "B" : self.best[-3:].reshape(3,)
            }
        output=[]
        for x in images:
            params["A0"] = x
            # from input_layer to hidden_1
            params["Z1"] = np.dot(params["W1"], params["A0"]) + params["B"][0]
            params["A1"] = self.sigmoid(params["Z1"])
            # from hidden_1 to hidden_2
            params["Z2"] = np.dot(params["W2"], params["A1"])+ params["B"][1]
            params["A2"] = self.sigmoid(params["Z2"])
            # from hidden_2 to output_layer
            params["Z3"] = np.dot(params["W3"], params["A2"])+ params["B"][2]

            output = self.softmax(params["Z3"])

            prediction = np.argmax(output)
            print(prediction)
            self.arrayToImg(x)
        return  output

    def arrayToImg(self,x):
          imge_array = np.asfarray(x).reshape((28,28))
          plt.imshow(imge_array,cmap='Greys',interpolation='None')
          plt.show()

    def train(self):

        #save the best fitness of each iteration and its iteration

        self.fitness_time, self.time = [], []
        for t in range(self.epochs):
            self.update_position_1()
            self.update_position_2()
            self.fitness_time.append(self.getFitness(self.bestIndice))
            self.time.append(t)
            print(self.fitnesses)
            print("acuuracy=",np.max(self.fitnesses),"pour it",t)
            print("-------- next itération dans le mêms nest")
            if np.max(self.fitnesses) > 0.95:
                self.saveNests()
                exit()
        if self.plot:
            self.Fplot()
        return self.getFitness(self.bestIndice)

    def Fplot(self):
        # Tracer le graphe d'accuracy
        plt.plot(self.time, self.fitness_time)
        plt.title('Accuracy value per Iterations')
        plt.xlabel('Iteration')
        plt.ylabel('Accuracy value')
        plt.show()


def make_shuffled_train(path):
    train_file = open(path, 'r')
    train_list = train_file.readlines()
    tmp_x_train=[]
    for x in train_list:
        tmp=x.split(',')
        tmp_x_train.append(tmp)
    train=np.asfarray(tmp_x_train)
    np.random.shuffle(train)
    tmp_x_train=[]
    tmp_y_train=[]
    for x in train:
        tmp_x_train.append(x[1:])
        tmp_y_train.append(x[:1])
    x_train=np.array(tmp_x_train)
    y_train=np.array(tmp_y_train).flatten()
    return x_train, y_train

def fromImgToNpArray(imgPath,imgType):
    lsData=[]
    for image_path in glob.glob(imgPath+"/*."+imgType):
        image = Image.open(image_path)
        imageToMatrice = np.asarray(image).flatten()
        lsData.append(imageToMatrice)
    return np.array(lsData)
x_train,y_train=make_shuffled_train("/content/drive/MyDrive/CS_DataSet/mnist_train.csv")



x_train = x_train/ 255
y_train = y_train.astype(int)
y_train = np.eye(10)[y_train]
#training
cs=CSO(x_train,y_train)

In [ ]:
class csa:


  def __init__(self,nb_nests, n_generations, min, max):
    self.nb_nests=nb_nests
    self.n_generations=n_generations
    self.min=min
    self.max=max

  def fitness_function(self,nest):
      #changer l'architectur pour un ANN opimized by cuckoo search
      cso=CSO(x_train,y_train)
      #meme changement pour mnist_momoentum
      #ann = DNN(nest)
      print("nest accuracy pour un nest")
      accuracy = cso.train()
      return accuracy

  def generate_sizes(self,min_value, max_value):
      return [random.randint(min_value, max_value) for _ in range(2)]

  def initnests(self):
      nests=[]
      for i in range(self.nb_nests) :
        nests.append(self.generate_sizes(self.min,self.max))
      return np.array(nests)


  def optimum(self,newnest,indiceOldnest):
              newFitness=self.fitness_function(newnest)
              oldFitness=self.fitness[indiceOldnest]
              bestnest=self.nests[indiceOldnest]
              if oldFitness < newFitness:
                  bestnest = newnest.copy()
                  #update fitness ?? discuter m3a leila
                  self.fitness[indiceOldnest]=newFitness
              return bestnest
  def cuckoo_search(self):
      # Initialize the population of network architectures
      nest_size = 2 # Replace this with the number of parameters you want to optimize

      self.nests=self.initnests()
      self.fitness = np.zeros(self.nb_nests)

      for i in range(self.nb_nests):
          self.fitness[i] = self.fitness_function(self.nests[i, :])


      print("fintesses initiales :",self.fitness)
      # Cuckoo Search algorithm
      newnests=self.nests.copy()
      for generation in range(self.n_generations):
          step_range=np.random.randint(0,10)
          for i in range(self.nb_nests):
              # Random walk
            if np.random.rand() < 0.75:
                step = np.random.randint(-step_range, step_range + 1,2)
                newnest=newnests[i] + step
                self.nests[i, :] = self.optimum(newnest,i)
            else:
                  # Abandon a solution with poor fitness score
                  j = np.argmin(self.fitness)
                  newnest=self.generate_sizes(self.min,self.max)
                  self.nests[i, :] = self.optimum(newnest,j)

          # Evaluate the fitness (accuracy) of the new solutions

          print("new fitness :",self.fitness)

      # Find the best solution
      i = np.argmax(self.fitness)
      best_nest = self.nests[i, :]
      best_accuracy = self.fitness[i]
      print(self.fitness)
      return best_nest, best_accuracy


cs=csa(3,3,1,64)
best_bird,bestAcurracy=cs.cuckoo_search()
print(best_bird,bestAcurracy)